In [10]:
import requests
import pandas as pd 

In [2]:
url = "https://api.climateview.global/boards/Boards/ec2d0cdf-e70e-43fb-85cb-ed6b31ee1e09/published"
r = requests.get(url)
r.json()

{'contentType': 'PublishedBoardVersion',
 'formatVersion': 2,
 'content': {'versionData': {'id': 'bcdcd0e4-f9b6-4348-85a8-f2050bb6755a',
   'boardId': 'ec2d0cdf-e70e-43fb-85cb-ed6b31ee1e09',
   'createdAt': 1677075780852,
   'publishedAt': 1677075780852},
  'boardData': {'boardGuid': 'ec2d0cdf-e70e-43fb-85cb-ed6b31ee1e09',
   'initiatingOrg': 'bf25a329-093a-42bb-96ce-c2e1cd6b519f',
   'board': {'id': 'ec2d0cdf-e70e-43fb-85cb-ed6b31ee1e09',
    'version': 2,
    'title': 'Panorama',
    'boardSettings': {'internationalisation': {'defaultLanguage': 'sv-PA'},
     'uploadImageForActionsEnabled': True,
     'disableTTEditing': True,
     'feedbackEmail': 'panorama@klimatpolitiskaradet.se',
     'surveyEmail': '',
     'suggestionEmail': 'panorama@klimatpolitiskaradet.se',
     'memberships': None,
     'auxiliaryActionProperties': [],
     'imgURL': 'https://maplauncher.blob.core.windows.net/b2c/Communities/Klimatlaget/Emissions/Vaxthusgasutslapp-totalt.svg',
     'displayUnit': 'weightMet

In [5]:
json_data = r.json()
json_data.keys()

dict_keys(['contentType', 'formatVersion', 'content'])

In [18]:
# parse indicator data
indicators = []
for d in json_data["content"]["boardData"]["indicators"]:
    item = {
        "id": d["id"],
        "title": d["title"],
        **d["indicatorProperties"],
    }
    indicators.append(item)

df = pd.DataFrame(indicators)

# Harmonisera enhet
# En del indikatorer är ton 
df["co2ePotential"] = df["co2ePotential"] * df["co2eUnit"].apply(lambda x: {"Mton": 1, "ton": 1 / 10e6}.get(x))

df.loc[70].head(30)


id                                     5fd0294d-0f23-4bbe-8989-27cd0f011a62
title                                   Minskad användning av lösningsmedel
type                                                                      3
diff                                                                    -92
potentialDescription      Diagrammet visar förändringen av utsläpp av vä...
description                                                             NaN
outcomeDescription        http://www.naturvardsverket.se/Sa-mar-miljon/S...
assessmentDescription                                                      
goalDescription           Grafen visar omställningens potentiella utveck...
imgSource                                                              Data
graphStartingYear                                                      2010
startingYear                                                           2010
endYear                                                                2045
potentialLeg

In [34]:


def get_outcome_data(l):
    s = pd.DataFrame(l).set_index("date")["values"]
    s.index = pd.to_datetime(s.index)
    # [ 2.2 ] = >2.2
    s = s.str[0].dropna()

    if len(s) > 0:
        resp = {
            "latest_value": s.iloc[-1],
            "latest_timepoint": s.index[-1],
        }
        if len(s) > 1:
            resp["prev_value"] = s.iloc[-2]
            resp["prev_timepoint"] = s.index[-2]
    else:
        resp = {}


    return pd.Series(resp)


df.outcome.apply(get_outcome_data)


/tmp/ipykernel_86145/3220883247.py:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series(resp)


,latest_value,latest_timepoint,prev_value,prev_timepoint
0,8.980,2020-01-01 00:00:00+00:00,11.720,2019-01-01 00:00:00+00:00
1,4.360,2020-01-01 00:00:00+00:00,3.450,2019-01-01 00:00:00+00:00
2,6.600,2020-01-01 00:00:00+00:00,10.700,2019-01-01 00:00:00+00:00
3,NaN,NaT,NaN,NaT
4,2.102,2021-01-01 00:00:00+00:00,2.116,2020-01-01 00:00:00+00:00
...,...,...,...,...
71,NaN,NaT,NaN,NaT
72,NaN,NaT,NaN,NaT
73,3.490,2019-01-01 00:00:00+00:00,3.610,2018-01-01 00:00:00+00:00
74,NaN,NaT,NaN,NaT


In [39]:
df_ts = pd.DataFrame(df.iloc[0].potentialCurve).set_index("date")
df_ts.index = pd.to_datetime(df_ts.index)
df_ts["value"] = df_ts["value"].interpolate()
df_ts["co2e"] = df_ts["co2e"].interpolate()
df_ts


,value,co2e
date,,
2010-01-01 00:00:00+00:00,10.260,0.000000
2011-01-01 00:00:00+00:00,10.466,0.024679
2012-01-01 00:00:00+00:00,10.672,0.049357
2013-01-01 00:00:00+00:00,10.878,0.074036
2014-01-01 00:00:00+00:00,11.084,0.098714
2015-01-01 00:00:00+00:00,11.290,0.123393
2016-01-01 00:00:00+00:00,11.496,0.148071
2017-01-01 00:00:00+00:00,11.702,0.172750
2018-01-01 00:00:00+00:00,11.908,0.197428
